---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
    # Load data into dataframe (use tab sepparation and skip the first line):
    data = pd.read_csv('Employee_Movie_Choices.txt', sep='\t', header=None, skiprows=1, names=['Employee', 'Movie'])
    # Convert dataframe to graph and return:
    return nx.from_pandas_dataframe(data, 'Employee', 'Movie')
# Testing:
#answer_one().edges()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    # Get employee movie graph:
    emp_mov_graph = answer_one()
    # Loop over sets to create node attibutes:
    for movie in movies:
        emp_mov_graph.node[movie]["type"] = "movie"
    for employee in employees:
        emp_mov_graph.node[employee]["type"] = "employee"
    # Return graph with node attributes:
    return emp_mov_graph
# Testing:
#answer_two().nodes(data=True)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
    # Make projection graph for employees:
    return bipartite.weighted_projected_graph(answer_two(), employees)
# Testing:
#answer_three().nodes()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [21]:
def answer_four():
    # Load employee relation edge list to dataframe:
    rel_df = pd.read_csv('Employee_Relationships.txt', delimiter="\t", header=None, names=["Employee 1", "Employee 2", "Relation Score"])
    
    # Load projection graph:
    proj_graph = answer_three()
    # Convert graph to pandas dataframe, but as an edge list:
    proj_df = pd.DataFrame(proj_graph.edges(data=True), columns=["Employee 1", "Employee 2", "Common Movies"])
    # Keep only the desired number from the dictionary returned as data:
    proj_df["Common Movies"] = proj_df["Common Movies"].apply(lambda x: x['weight'])

    # To merge both dataframes, we must have the according pairs of Employee 1 and 2.
    # Therefore, we will mirror the employees column of our projection. This will
    # give us duplicate data, but must be used to merge the dataframes.
    
    # Switch columns:
    switch_proj_df = proj_df[["Employee 2", "Employee 1", "Common Movies"]]
    # Rename columns to match original dataframe:
    switch_proj_df.columns = proj_df.columns.tolist()
    # Concatenate on original projection dataframe:
    full_proj_df = pd.concat([proj_df, switch_proj_df])
    
    # Merge this full projection dataframe on the relation dataframe:
    merged_df = pd.merge(rel_df, full_proj_df, how='left', on=["Employee 1", "Employee 2"])
    # Fill missing values with 0:
    merged_df.fillna(0, inplace=True)
    
    # Now we can obtain correlation between Relation Score and Common Movies without using
    # the specific names as features (as would happen if we extracted adjacent matrices from the two graphs)!
    
    # Return the correlation between the two variables:
    return merged_df.corr()['Relation Score']['Common Movies']

# Testing:
answer_four()

0.78839622217334759